In [2]:
from datetime import datetime
from pathlib import Path
from typing import Iterator
import os
import mlflow
import torch
import torch.nn as nn
import torch.optim as optim
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from loguru import logger
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer import ReportTypes, Trainer, TrainerSettings, metrics
from mltrainer.preprocessors import BasePreprocessor


c:\Users\leons\OneDrive\Bureaublad\School\ML Trainer\.venv\lib\site-packages\pydantic\_internal\_fields.py:160: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\leons\OneDrive\Bureaublad\School\ML Trainer\.venv\lib\site-packages\pydantic\_internal\_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [3]:
def get_fashion_streamers(batchsize: int) -> tuple[Iterator, Iterator]:
    fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FLOWERS)
    preprocessor = BasePreprocessor()
    streamers = fashionfactory.create_datastreamer(
        batchsize=batchsize, preprocessor=preprocessor
    )
    train = streamers["train"]
    valid = streamers["valid"]
    len_train = len(train)
    len_valid = len(valid)
    trainstreamer = train.stream()
    validstreamer = valid.stream()
    return trainstreamer, validstreamer, len_train, len_valid

trainstreamer, validstreamer, len_train, len_valid = get_fashion_streamers(64)

2024-09-23 16:43:45.895 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\leons\.cache\mads_datasets\flowers


In [4]:
print(len_train, len_valid)

45 11


In [7]:
x, y = next(iter(trainstreamer))
print(x)

tensor([[[[0.9961, 0.9882, 0.9922,  ..., 0.1961, 0.2000, 0.2000],
          [0.9686, 0.9804, 0.9961,  ..., 0.2000, 0.2000, 0.2000],
          [0.9804, 0.9765, 0.9804,  ..., 0.2039, 0.2039, 0.2039],
          ...,
          [0.9804, 0.9765, 0.9725,  ..., 0.9843, 0.9804, 0.9882],
          [0.9882, 0.9843, 0.9804,  ..., 0.9922, 0.9843, 0.9804],
          [0.9843, 0.9804, 0.9804,  ..., 0.9882, 0.9922, 0.9725]],

         [[0.9529, 0.9569, 0.9647,  ..., 0.4353, 0.4392, 0.4392],
          [0.9176, 0.9373, 0.9686,  ..., 0.4392, 0.4392, 0.4392],
          [0.9255, 0.9255, 0.9451,  ..., 0.4431, 0.4431, 0.4431],
          ...,
          [0.7725, 0.7765, 0.7686,  ..., 0.8314, 0.8000, 0.7922],
          [0.7765, 0.7804, 0.7882,  ..., 0.9059, 0.8549, 0.8314],
          [0.7725, 0.7765, 0.7882,  ..., 0.9529, 0.9216, 0.8706]],

         [[0.0235, 0.0314, 0.0431,  ..., 0.6510, 0.6549, 0.6549],
          [0.0000, 0.0078, 0.0353,  ..., 0.6549, 0.6549, 0.6549],
          [0.0157, 0.0078, 0.0078,  ..., 0